Import libraries

In [37]:
import pandas as pd
from faker import Faker
import random
from functools import reduce

In [38]:
def format_harga(harga):
    return f"IDR {harga:,.0f}".replace(",", ".")

1. Buatlah sebuah daftar atau list ‘data_penjualan’ yang berisi informasi seperti 
IDproduk, nama produk, harga, jumlah terjual (quantity), dan tanggal penjualan yang 
disusun dalam struktur data dictionary.

In [39]:
def generate_data(num_days=4, products_per_day=3):
    faker = Faker()
    dates = pd.date_range("2024-08-06", periods=num_days).strftime('%Y-%m-%d')

    # Use faker to generate product names and simulate food-related names
    food_names = [f"{faker.word()} {random.choice(['Pizza', 'Burger', 'Pasta', 'Salad', 'Sushi', 'Soup', 'Steak', 'Sandwich'])}" 
                  for _ in range(products_per_day * num_days)]
    
    # list comprehension
    data_penjualan = pd.DataFrame({
        "id": ["GN" + str(random.randint(100, 999)) for _ in range(products_per_day * num_days)],
        "nama": food_names,
        "harga": [random.randint(20000, 200000) for _ in range(products_per_day * num_days)],
        "quantity": [random.randint(1, 100) for _ in range(products_per_day * num_days)],
        "tanggal": [date for date in dates for _ in range(products_per_day)]
    })

    # map
    data_penjualan['harga_formatted'] = list(map(format_harga, data_penjualan['harga']))

    return data_penjualan

# Call the function to get the data
data_penjualan = generate_data()
print(data_penjualan[['id', 'nama', 'harga_formatted', 'quantity', 'tanggal']])

       id              nama harga_formatted  quantity     tanggal
0   GN546  especially Sushi     IDR 136.605        27  2024-08-06
1   GN677     city Sandwich     IDR 143.029        11  2024-08-06
2   GN731       visit Steak     IDR 137.722        80  2024-08-06
3   GN682         of Burger      IDR 81.731        19  2024-08-07
4   GN494         view Soup     IDR 163.528        65  2024-08-07
5   GN991         rule Soup     IDR 170.434         3  2024-08-07
6   GN573          you Soup      IDR 94.266        61  2024-08-08
7   GN937         lot Pasta     IDR 127.519        64  2024-08-08
8   GN469        ever Salad      IDR 22.114        88  2024-08-08
9   GN108     surface Salad     IDR 197.441        43  2024-08-09
10  GN353   effect Sandwich      IDR 81.435         5  2024-08-09
11  GN655      effort Salad     IDR 137.444        18  2024-08-09


2. Kemudian buatlah fungsi ‘hitung_pendapatan’ yang mengembalikan daftar baru dari semua informasi produk yang disertai total pendapatan untuk setiap produk.

In [40]:
def hitung_pendapatan(df):
    # Vectorized operation in Pandas for calculating revenue
    df['pendapatan'] = df['harga'] * df['quantity']
    df['pendapatan_formatted'] = df['pendapatan'].apply(lambda x: format_harga(int(x)))
    return df[['id', 'nama', 'harga_formatted', 'quantity', 'tanggal', 'pendapatan', 'pendapatan_formatted']]

data_with_pendapatan = hitung_pendapatan(data_penjualan)
print(data_with_pendapatan.to_string(index=False)) 

   id             nama harga_formatted  quantity    tanggal  pendapatan pendapatan_formatted
GN546 especially Sushi     IDR 136.605        27 2024-08-06     3688335        IDR 3.688.335
GN677    city Sandwich     IDR 143.029        11 2024-08-06     1573319        IDR 1.573.319
GN731      visit Steak     IDR 137.722        80 2024-08-06    11017760       IDR 11.017.760
GN682        of Burger      IDR 81.731        19 2024-08-07     1552889        IDR 1.552.889
GN494        view Soup     IDR 163.528        65 2024-08-07    10629320       IDR 10.629.320
GN991        rule Soup     IDR 170.434         3 2024-08-07      511302          IDR 511.302
GN573         you Soup      IDR 94.266        61 2024-08-08     5750226        IDR 5.750.226
GN937        lot Pasta     IDR 127.519        64 2024-08-08     8161216        IDR 8.161.216
GN469       ever Salad      IDR 22.114        88 2024-08-08     1946032        IDR 1.946.032
GN108    surface Salad     IDR 197.441        43 2024-08-09     848996

3. Buatlah fungsi ‘average_penjualan’ yang akan mengembalikan rata-rata penjualan di tanggal tertentu sesuai inputan user. Jangan lupa buatlah exception handling apabila tanggal tidak ditemukan atau tidak sesuai format.

In [41]:
def average_penjualan(df, tanggal, minRevenue=50000):
    try:
        sales_on_date = df[df['tanggal'] == tanggal]
        if sales_on_date.empty:
            raise ValueError("Tanggal tidak ditemukan.")
        
        mewing = sales_on_date[sales_on_date['pandapatan'] >= minRevenue]
        
        if mewing.empty:
            return f"mewing skibidi sigma Ohio {format_harga(minRevenue)} fanum tax."
            
        fanum = reduce(int.__add__, (mewing['pendapatan']))
        # # Compute average using pandas operations
        # avg_sales = (sales_on_date['harga'] * sales_on_date['quantity']).mean()
        # return format_harga(int(avg_sales))
        
        rata_rata = fanum / len(mewing)
        return format_harga (int(rata_rata))
    
    except ValueError as e:
        return str(e)
    except Exception as e:
        return "Format tanggal tidak sesuai."

# Example usage
tanggal_input = input("skibidi")
average = average_penjualan(data_penjualan, tanggal_input)
print(f"Rata-rata penjualan pada tanggal {tanggal_input}: {average}")

Rata-rata penjualan pada tanggal 2024-08-07: Format tanggal tidak sesuai.


4. Buatlah fungsi ‘total_penjualan’ sebagai generator dengan inputan data hasil ‘hitung_pendapatan’ yang akan menghasilkan penjualan per tanggal pada setiap produk. Contoh output:

In [42]:
def total_penjualan(df):
    grouped = df.groupby('tanggal')['pendapatan'].sum().reset_index()
    for _, row in grouped.iterrows():
        yield {"tanggal": row['tanggal'], "total_penjualan": format_harga(int(row['pendapatan']))}

data_with_pendapatan = hitung_pendapatan(data_penjualan)
for penjualan in total_penjualan(data_with_pendapatan):
    print(f"Tanggal: {penjualan['tanggal']}, Total Penjualan: {penjualan['total_penjualan']}")


Tanggal: 2024-08-06, Total Penjualan: IDR 16.279.414
Tanggal: 2024-08-07, Total Penjualan: IDR 12.693.511
Tanggal: 2024-08-08, Total Penjualan: IDR 15.857.474
Tanggal: 2024-08-09, Total Penjualan: IDR 11.371.130
